In [3]:
import torch
from torch_geometric.data import Data
import networkx as nx

In [4]:
graphs = []

ngraphs = 10000
for idx in range(ngraphs):
    n = 10 # Graph size
    g = nx.path_graph(n) # Path graph
    edge_index = torch.tensor(list(g.edges())).t().contiguous()

    # Data and labels are mirrors
    x = torch.randperm(n)
    y = torch.flip(x, [0])
    x = x.to(dtype=torch.float32).unsqueeze(dim=1)

    # Mask
    k = n//10
    mask = torch.cat((torch.ones(k), -torch.ones(k), torch.zeros(n - 2 * k)))
    mask = mask[torch.randperm(n)]
    train_mask = mask == 0
    val_mask = mask == 1
    test_mask = mask == -1

    val_mask[(idx+1)%3] = 1
    test_mask[(idx+2)%3] = 1
    train_mask[val_mask | test_mask] = 0

    # Create the graph
    data = Data(edge_index=edge_index, x=x, y=y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
    graphs.append(data)

In [5]:
import pickle

fname = "mirror.pkl"

with open(fname, "wb") as f:
    pickle.dump(graphs, f)

In [ ]:
import hashlib

def calculate_md5(file_path):
    # Create an MD5 hash object
    md5 = hashlib.md5()

    # Open the file in binary mode
    with open(file_path, "rb") as file:
        # Read the file in chunks
        chunk_size = 8192
        for chunk in iter(lambda: file.read(chunk_size), b""):
            # Update the hash with the current chunk
            md5.update(chunk)

    # Return the hexadecimal representation of the digest
    return md5.hexdigest()

In [ ]:
from torch_geometric.data import InMemoryDataset, download_url
import os
import os.path as osp
import shutil
import fsspec
import io

def torch_save(data, path) -> None:
    buffer = io.BytesIO()
    torch.save(data, buffer)
    with fsspec.open(path, 'wb') as f:
        f.write(buffer.getvalue())

def torch_load(path):
    with fsspec.open(path, 'rb') as f:
        return torch.load(f)

class Mirror(InMemoryDataset):
    def __init__(self):
        super().__init__('/tmp/Mirror')
        path = osp.join(self.processed_dir, self.processed_file_names[0])
        self.load(path)

    def download(self):
        src = "/nobackup/vbalivada/GraphGPS/cs762/mirror.pkl"
        dst = osp.join(self.raw_dir, "mirror.pkl")
        shutil.copy(src, dst)

    @property
    def raw_file_names(self):
        return ['mirror.pkl']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        with open(osp.join(self.raw_dir, "mirror.pkl"), "rb") as f:
            graphs = pickle.load(f)
        self.save(self.__class__, graphs, osp.join(self.processed_dir, self.processed_file_names[0]))

    @property
    def processed_file_names(self):
        return ["data.pt"]

    @staticmethod
    def save(cls, data_list, path):
        r"""Saves a list of data objects to the file path :obj:`path`."""
        data, slices = cls.collate(data_list)
        torch_save((data.to_dict(), slices), path)
    
    def load(self, path):
        r"""Loads the dataset from the file path :obj:`path`."""
        data, self.slices = torch_load(path)
        if isinstance(data, dict):  # Backward compatibility.
            data = Data.from_dict(data)
        self.data = data

In [ ]:
mirror_dataset = Mirror()
g = mirror_dataset[0]

In [ ]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.embedding = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        if src_mask is None:
            """Generate a square causal mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
            """
            src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0), 0, :]
        return self.dropout(x)

In [ ]:
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
model = TransformerModel(ntoken=10, d_model=8, nhead=2, d_hid=8, nlayers=2, dropout=0.1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    for data in mirror_dataset:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x.squeeze().to(torch.long))
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()    
    print(loss.item())

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    for data in dataset:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

In [ ]:
pred[data.test_mask], data.y[data.test_mask]